## BICEP CURL COUNTER

In [1]:
# dependency for live vedio feed 
import cv2 

## Mediapipe detection in live video

In [2]:
# dependency for detection 
import mediapipe as mp 
mp_pose = mp.solutions.pose 
mp_draw = mp.solutions.drawing_utils 

In [5]:
def mediapipe_detection():
    cap = cv2.VideoCapture(0)
    with mp_pose.Pose() as pose: 
        while cap.isOpened():
            _, frame = cap.read() 

            # change color from bgr to rgb 
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            result = pose.process(image)                             # returns result object with landmarks details

            # drawing landmarks using details from result object
            mp_draw.draw_landmarks(frame, result.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_draw.DrawingSpec(color=(255,0,0), thickness=2, circle_radius=2),
                                  mp_draw.DrawingSpec(color=(255,0,0), thickness=2, circle_radius=2))
            cv2.imshow('Detected image', frame)

            if cv2.waitKey(10) & 0xFF == ord('q'):                   # press q to stop live feed 
                break
        
        cap.release()
        cv2.destroyAllWindows()
        
    return result

## Extracting necessary points or landmarks from the detected image 

In [6]:
# necessary points for counting bicep curl are 
# i)   shoulder point 
# ii)  elbow point 
# iii) wrist point

result = mediapipe_detection()
landmark = result.pose_landmarks.landmark

In [7]:
len(landmark)

33

In [8]:
mp_pose.PoseLandmark.LEFT_SHOULDER.value              # index for left shoulder in landmark list

11

In [9]:
landmark[mp_pose.PoseLandmark.LEFT_SHOULDER.value]   # co-ordinates for left shoulder

x: 0.649353
y: 0.3018976
z: -0.16498305
visibility: 0.99875444

In [10]:
mp_pose.PoseLandmark.LEFT_ELBOW.value              # index for left elbow in landmark list

13

In [11]:
landmark[mp_pose.PoseLandmark.LEFT_ELBOW.value]   # co-ordinates for left elbow

x: 0.6934205
y: 0.61739045
z: -0.014485715
visibility: 0.86901474

In [12]:
mp_pose.PoseLandmark.LEFT_WRIST.value              # index for left wrist in landmark list 

15

In [13]:
landmark[mp_pose.PoseLandmark.LEFT_WRIST.value]   # co-ordinates for left wrist

x: 0.7690108
y: 0.86094964
z: -0.12368449
visibility: 0.7062197

## Calculating angles for curl count 

In [16]:
# dependency for calculating angles
import numpy as np

In [17]:
def calculate_angle(a, b, c):
    a = np.array(a)        # a is shoulder
    b = np.array(b)        # b is elbow
    c = np.array(c)        # c is wrist
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angles = np.abs((radians * 180)/np.pi)
    
    if angles > 180:
        angles = 360 - angles
        
    return angles

In [18]:
shoulder_cords = [landmark[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmark[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
print(shoulder_cords)

elbow_cords = [landmark[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmark[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
print(elbow_cords)

wrist_cords = [landmark[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmark[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
print(wrist_cords)

[0.64935302734375, 0.3018975853919983]
[0.6934205293655396, 0.6173904538154602]
[0.7690107822418213, 0.8609496355056763]


In [19]:
angle = calculate_angle(shoulder_cords, elbow_cords, wrist_cords)
print(angle)

170.70944479116955


## Visualizing the virtual bicep-curl counter 

In [45]:
def bicep_curl_count():
    count = 0
    count_r = 0
    stage = None

    cap = cv2.VideoCapture(0)
    with mp_pose.Pose() as pose: 
        while cap.isOpened():
            _, frame = cap.read() 

            # change color from bgr to rgb 
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            result = pose.process(image)                             # returns result object with landmarks details

            try:
                landmark = result.pose_landmarks.landmark

                # extracting landmarks for left hand
                shoulder_cords = [landmark[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmark[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                elbow_cords = [landmark[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmark[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                wrist_cords = [landmark[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmark[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                
                
                # extracting landmarks for right hand
                shoulder_cords_r = [landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                elbow_cords_r = [landmark[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmark[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
                wrist_cords_r = [landmark[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmark[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
     
                
                # calculating bicep-curl angle
                bicep_angle = calculate_angle(shoulder_cords, elbow_cords, wrist_cords)
                bicep_angle_r = calculate_angle(shoulder_cords_r, elbow_cords_r, wrist_cords_r)

                # writing bicep_curl angle in the live frame 
                '''cv2.putText(frame, "calculated angle"+str(bicep_angle_r), (40,40), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 2, cv2.LINE_AA )'''

                
                # left bicep-curl counting
                if(bicep_angle > 160): 
                    stage = "down"
                    bicep_angle = 160
                elif(bicep_angle < 15 and stage =="down"):
                    stage = "up"
                    count += 1 
                    bicep_angle = 15
                else:
                    bicep_angle = bicep_angle - 15
                    
                percen = (bicep_angle/145.0)*100
                
                val = int(percen)
                if val > 100:
                    val = 100
                    cv2.rectangle(frame, (60, 100+val), (80, 200), (0,255,0), -1) 
                elif val < 0:
                    val = 0
                    cv2.rectangle(frame, (60, 100+val), (80, 200), (0,255,0), -1) 
                else:
                    cv2.rectangle(frame, (60, 100+val), (80, 200), (0,0,255), -1)
                    
                cv2.rectangle(frame, (60,100), (80, 200), (0,0,0), 2)
                
                cv2.putText(frame, "count "+str(count), (40, 250), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 2, cv2.LINE_AA)
                                
                
                # right bicep-curl counting
                if(bicep_angle_r > 160): 
                    stage_r = "down"
                    bicep_angle_r = 160
                elif(bicep_angle_r < 15 and stage_r =="down"):
                    stage_r = "up"
                    count_r += 1 
                    bicep_angle_r = 15
                else:
                    bicep_angle_r = bicep_angle_r - 15
                    
                percent = (bicep_angle_r/145.0)*100
                
                val_r = int(percent)
                if val_r > 100:
                    val_r = 100
                    cv2.rectangle(frame, (560, 100+val_r), (580, 200), (0,255,0), -1) 
                elif val_r < 0:
                    val_r = 0
                    cv2.rectangle(frame, (560, 100+val_r), (580, 200), (0,255,0), -1) 
                else:
                    cv2.rectangle(frame, (560, 100+val_r), (580, 200), (0,0,255), -1)
                    
                cv2.rectangle(frame, (560,100), (580, 200), (0,0,0), 2)
                
                cv2.putText(frame, "count "+str(count_r), (540, 250), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 2, cv2.LINE_AA)
                    
                
            except:
                pass

            # drawing landmarks using details from result object
            mp_draw.draw_landmarks(frame, result.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_draw.DrawingSpec(color=(255,0,0), thickness=2, circle_radius=2),
                                  mp_draw.DrawingSpec(color=(255,0,0), thickness=2, circle_radius=2))
            cv2.imshow('Bicep image', frame)

            if cv2.waitKey(10) & 0xFF == ord('q'):                   # press q to stop live feed 
                break

        cap.release()
        cv2.destroyAllWindows()

In [47]:
bicep_curl_count()